In [1]:
from ctapipe.core import Container, Item, Map
import numpy as np

In [25]:
class SubContainer(Container):
    junk = Item("nothing","Some junk")

class EventContainer(Container):
    event_id = Item(-1,"event id number")
    tels_with_data = Item([], "list of telescopes with data")
    sub = Item(SubContainer(), "stuff")  # a sub-container in the hierarchy

    # for dicts of sub-containers, use Map instead 
    # of a dict() as the default value to support serialization
    tel = Item(Map(), "telescopes")  


In [26]:
ev = EventContainer()

default values automatically filled in

In [27]:
print(ev.event_id)
print(ev.tels_with_data)
print(ev.tel)

-1
[]
{}


In [28]:
print(ev)

{'_metadata': {},
 'event_id': -1,
 'sub': {'_metadata': {}, 'junk': 'nothing'},
 'tel': {},
 'tels_with_data': []}


values can be set as normal for a class:

In [6]:
ev.event_id = 100
ev.event_id

100

In [7]:
ev.as_dict()  # by default only shows the bare items, not sub-containers (See later)

{'_metadata': {},
 'event_id': 100,
 'sub': <__main__.SubContainer at 0x109bd6f98>,
 'tel': {},
 'tels_with_data': []}

In [8]:
ev.as_dict(recursive=True)

{'_metadata': {},
 'event_id': 100,
 'sub': {'_metadata': {}, 'junk': 'nothing'},
 'tel': {},
 'tels_with_data': []}

Now, let's define a sub-container that we can add per telescope:

In [9]:
class TelContainer(Container):
    tel_id = Item(-1, "telescope ID number")
    image = Item(np.zeros(10), "camera pixel data")



and we can add a few of these to the parent container inside the tel dict:

In [10]:
ev.tel[10] = TelContainer()
ev.tel[5] = TelContainer()
ev.tel[42] = TelContainer()

In [11]:
ev.tel

{5: <__main__.TelContainer at 0x109be04e0>,
 10: <__main__.TelContainer at 0x109be04a8>,
 42: <__main__.TelContainer at 0x109be0550>}

### converion to dictionaries

In [19]:
ev.as_dict()

{'_metadata': {},
 'event_id': 100,
 'sub': <__main__.SubContainer at 0x109bd6f98>,
 'tel': {5: <__main__.TelContainer at 0x109be04e0>,
  10: <__main__.TelContainer at 0x109be04a8>,
  42: <__main__.TelContainer at 0x109be0550>},
 'tels_with_data': []}

In [20]:
ev.as_dict(recursive=True, flatten=False)

{'_metadata': {},
 'event_id': 100,
 'sub': {'_metadata': {}, 'junk': 'nothing'},
 'tel': {5: {'_metadata': {},
   'image': array([ 9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.]),
   'tel_id': -1},
  10: {'_metadata': {},
   'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
   'tel_id': -1},
  42: {'_metadata': {},
   'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
   'tel_id': -1}},
 'tels_with_data': []}

for serialization to a table, we can even flatten the output into a single set of columns

In [21]:
ev.as_dict(recursive=True, flatten=True)

{'_metadata': {},
 'event_id': 100,
 'sub__metadata': {},
 'sub_junk': 'nothing',
 'tel_10': {'_metadata': {},
  'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  'tel_id': -1},
 'tel_42': {'_metadata': {},
  'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
  'tel_id': -1},
 'tel_5': {'_metadata': {},
  'image': array([ 9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.]),
  'tel_id': -1},
 'tels_with_data': []}

### setting and clearing values

In [23]:
ev.tel[5].image[:] = 9
print(ev)

{'_metadata': {},
 'event_id': 100,
 'sub': {'_metadata': {}, 'junk': 'nothing'},
 'tel': {5: {'_metadata': {},
             'image': array([ 9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.,  9.]),
             'tel_id': -1},
         10: {'_metadata': {},
              'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
              'tel_id': -1},
         42: {'_metadata': {},
              'image': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
              'tel_id': -1}},
 'tels_with_data': []}


In [24]:
ev.reset()
ev.as_dict(recursive=True, flatten=True)

{'_metadata': {},
 'event_id': -1,
 'sub__metadata': {},
 'sub_junk': 'nothing',
 'tels_with_data': []}